### Inspired by:

https://github.com/lutraconsulting/MDAL/issues/155

see python script in-line from Marcelo Andrioni (https://github.com/marceloandrioni)

In [2]:
import sys
import argparse
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import xarray as xr

In [3]:
base_path = '/datasets/work/oa-vankirap/work/'
output_path = f'{base_path}/schism/EVA/'

In [5]:
# ds_schism= xr.open_dataset(f'{base_path}schism/EVA/Hindcast_v3_hs_POT_GPD_ARI_20230718.nc')
# try with an actual simulation output, rather than derived file ...
base_path = f'/datasets/work/oa-roamsurf/work'
schism_out_path = f'{base_path}/airflow_beta/results/schism_TS'
fc_time = '2023-08-02T120000'
schism_path = f'{schism_out_path}/{fc_time}/schout_1.nc'
ds_schism= xr.open_dataset(schism_path)

ds_schism

<xarray.Dataset>
Dimensions:                  (time: 96, one: 1, nSCHISM_hgrid_face: 305131,
                              nMaxSCHISM_hgrid_face_nodes: 4,
                              nSCHISM_hgrid_edge: 467518, two: 2,
                              nSCHISM_hgrid_node: 162298, sigma: 2,
                              nSCHISM_vgrid_layers: 2)
Coordinates:
  * time                     (time) datetime64[ns] 2023-08-01T13:00:00 ... 20...
  * sigma                    (sigma) float32 -1.0 0.0
Dimensions without coordinates: one, nSCHISM_hgrid_face,
                                nMaxSCHISM_hgrid_face_nodes,
                                nSCHISM_hgrid_edge, two, nSCHISM_hgrid_node,
                                nSCHISM_vgrid_layers
Data variables: (12/34)
    SCHISM_hgrid             (one) int32 ...
    SCHISM_hgrid_face_nodes  (nSCHISM_hgrid_face, nMaxSCHISM_hgrid_face_nodes) float64 ...
    SCHISM_hgrid_edge_nodes  (nSCHISM_hgrid_edge, two) int32 ...
    SCHISM_hgrid_node_x      (nSCHISM_hgrid_node) float64 ...
    SCHISM_hgrid_node_y      (nSCHISM_hgrid_node) float64 ...
    node_bottom_index        (nSCHISM_hgrid_node) int32 ...
    ...                       ...
    WWM_2                    (time, nSCHISM_hgrid_node) float32 ...
    WWM_10                   (time, nSCHISM_hgrid_node) float32 ...
    WWM_11                   (time, nSCHISM_hgrid_node) float32 ...
    WWM_18                   (time, nSCHISM_hgrid_node) float32 ...
    WWM_19                   (time, nSCHISM_hgrid_node) float32 ...
    WWM_20                   (time, nSCHISM_hgrid_node) float32 ...
Attributes:
    Conventions:   CF-1.0, UGRID-1.0
    title:         SCHISM Model output
    institution:   SCHISM Model output
    source:        SCHISM model output version v10
    references:    http://ccrm.vims.edu/schismweb/
    history:       created by combine_output11
    comment:       SCHISM Model output
    type:          SCHISM Model output
    VisIT_plugin:  https://schism.water.ca.gov/library/-/document_library/vie...

In [4]:
ds_schism = ds_schism.rename_dims('nSCHISM_hgrid_node':'node','nSCHISM_hgrid_face':,'nele','nvertex'

SyntaxError: invalid syntax (3380799984.py, line 1)

In [ ]:
ds_schism = ds_schism.rename({'SCHISM_hgrid_node_x': 'longitude',
                'SCHISM_hgrid_node_y': 'latitude',
                'SCHISM_hgrid_face_nodes': 'mesh_topology',
               'depth': 'bathymetry',})

In [ ]:
ds_schism['latitude'].attrs = {'long_name': 'latitude',
                        'standard_name': 'latitude',
                        'units': 'degrees_north'}

ds_schism['longitude'].attrs = {'long_name': 'longitude',
                         'standard_name': 'longitude',
                         'units': 'degrees_east'}

for var in ['latitude', 'longitude']:
    ds_schism[var].encoding = {'dtype': 'f4', '_FillValue': None}

In [6]:
ds_schism['bathymetry'].attrs = {'long_name': 'bathymetry',
                          'standard_name': 'sea_floor_depth',
                          'units': 'm'}

In [10]:
len(ds_schism.bathymetry.dims)

1

In [11]:
list(ds_schism.data_vars)

['mean',
 'max',
 'p99',
 'c',
 'scale',
 '10',
 '10 lower ci',
 '10 upper ci',
 '50',
 '50 lower ci',
 '50 upper ci',
 '100',
 '100 lower ci',
 '100 upper ci',
 'mesh_topology',
 'longitude',
 'latitude',
 'bathymetry']

In [13]:
for var in list(ds_schism.data_vars):
    if len(ds_schism[var].dims) == 1:
        ds_schism[var].attrs['location'] = 'node'
        ds_schism[var].attrs['mesh'] = 'mesh_topology'

    # # chunking for faster access
    # nnodes = ds_schism['node'].size
    # chunksize = [1, nnodes] if 'time' in list(ds_schism[var].coords_schism) else nnodes

    # ds_schism[var].encoding = {'dtype': 'f4',
    #                     '_FillValue': np.nan,
    #                     # 'zlib': True,
    #                     # 'complevel': 1,
    #                     'chunksize': chunksize}

In [15]:
# set some attributes to follow CF convetions for unstructured grids_schism
# https://github.com/ugrid-conventions/ugrid-conventions
ds_schism['mesh_topology'].attrs['node_coordinates'] = 'longitude latitude'
ds_schism['mesh_topology'].attrs['cf_role'] = 'mesh_topology'
# ds_schism['mesh_topology'].attrs['topology_dimension'] = \
#     ds_schism['mesh_topology'].attrs['dimension']

In [19]:
today = pd.Timestamp('today')

In [20]:
ds_schism.attrs = {'title': 'VanKIRAP National Coastal Hazard Model: Significant Wave Height Extreme Value Analysis (EVA: POT/GPD Fit), 10-, 50- and 100-year average return intervals',
            'institution': 'CSIRO/Van KIRAP',
            'source': 'SCHISM ()',
            'history': f'based on Hindcast_v3, created{today:%Y%m%d}',
            'references': 'https://research.csiro.au/slrwavescoast/',
            'Conventions': 'CF-1.7, UGRID-1.0'}

In [22]:
ds_schism.to_netcdf(f'{output_path}/Hindcast_v3_hs_POT_GPD_ARI_{today:%Y%m%d}_UGRID.nc')

In [25]:
ds_schism.latitude.values

array([-14.75888, -14.70902, -14.65916, ..., -17.60058, -17.55072,
       -17.50086])